In [125]:
import pandas as pd
import numpy as np

### やること
- いったん出すやつsubmissionのIDごとに処理
    - これまでの購入リストからIDで探索 そいつのこれまでのarticleID を取得
    - 購入した品物の index group ID の最頻値を決定
    - 年齢と最頻値からグループ決定，トップ１２位をそいつの予測とする
        - 年齢の欠損や，グループでの予測がないときは全体のトップ１２で代用する   - やること
-   

In [126]:
# DATA_DIR = "./temp_data/0320_final.pkl"
DATA_DIR = "./temp_data/0320_final_full.pkl"
topn_df = pd.read_pickle(DATA_DIR)


In [127]:
# データの読み込み
DATA_DIR = "../data/original/"
articles_df=pd.read_pickle(f'{DATA_DIR}articles.pkl')
customers_df=pd.read_pickle(f'{DATA_DIR}customers.pkl')
transactions_train_df=pd.read_pickle(f'{DATA_DIR}transactions_train.pkl')

sample_submission_df = pd.read_pickle(f'{DATA_DIR}sample_submission.pkl')

In [128]:
transactions_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   t_dat             object
 1   customer_id       object
 2   article_id        object
 3   price             object
 4   sales_channel_id  object
dtypes: object(5)
memory usage: 1.2+ GB


- submission ID ごとに
    - customerから年齢検索，10刻みの形に
    - transactionからarticleIDを探索，article使ってindex_group_noを取得

In [35]:
# articles_df = articles_df.head(10000)
# customers_df = customers_df.head(10000)
# transactions_train_df = transactions_train_df.head(10000)

In [129]:
# articles と customersをtransactionsを使って結合する
def create_features(articles_df,customers_df,transactions_train_df):
    tt_temp = transactions_train_df[["customer_id","article_id"]]
    a_df = articles_df[["article_id","index_group_no"]]
    c_df = customers_df[["customer_id","age"]]
    at_merged_df =  pd.merge(tt_temp,a_df)
    features_df = pd.merge(c_df,at_merged_df)
    features_df = features_df.dropna(how="all")
    return features_df

In [130]:
features_df = create_features(articles_df,customers_df,transactions_train_df)

In [131]:
# ageの欠損値を0で埋める
def fillna_to_zero_in_age(df):
    df["age"] = df["age"].fillna(0)
    return df

In [132]:
features_dfb = fillna_to_zero_in_age(features_df)

In [134]:
# 年齢を10ごとにする
def convert_age_into_10_delimited(df):
    df["age"] = df["age"].astype(np.int64) 
    df["age"] = df["age"] / 10
    df["age"] = df["age"].astype(np.int64) 
    df["age"] = df["age"] * 10
    return df
features_df = convert_age_into_10_delimited(features_df)

In [135]:
# customer_idごとにindex_group_noの最頻値を取る
def create_df_mode_index_group_no(df):
    df = df.groupby(["age","customer_id"])["index_group_no"].agg(lambda x:x.mode()[0]).reset_index()
    return df

In [136]:
mode_index_group_no_df = create_df_mode_index_group_no(features_df)

In [137]:
mode_index_group_no_df["index_group_no"] = mode_index_group_no_df["index_group_no"].astype("string")
topn_df["index_group_no"] = topn_df["index_group_no"].astype("string")

In [138]:
# mode_index_group_no_dfとtopn_dfの左側結合
merged_df = pd.merge(mode_index_group_no_df, topn_df, on=["age","index_group_no"], how="left")

In [139]:
# NANの最頻値保管
def fillna_topn(df):
    df = df.fillna(df.mode().iloc[0])
    return df

In [149]:
merged_topn_df = fillna_topn(merged_df.copy())

In [150]:

merged_topn_df.to_pickle("./temp_data/0417_full.pkl")

In [152]:
# sample　submissionの順番にしたいのでsample_submission_dfと左結合
# ageとindex_group_noをドロップ
customer_id_and_tops_df = merged_topn_df.drop(["age","index_group_no"],axis=1)

customer_id_df = sample_submission_df[["customer_id"]]
correct_customer_id_df  = pd.merge(customer_id_df, customer_id_and_tops_df, how='left')


In [118]:
# topsをジョインしてpredictionを作る＋いらないやつを消す

def join_tops(df):
        
    df["prediction"] = df['top1'].astype(str) + ' ' + df['top2'].astype(str)+ ' ' + df['top3'].astype(str)+ ' ' + df['top4'].astype(str)+ ' ' + df['top5'].astype(str)+ ' ' + df['top6'].astype(str)+ ' ' + df['top7'].astype(str) + ' ' + df['top8'].astype(str) + ' ' + df['top9'].astype(str)+ ' ' + df['top10'].astype(str) + ' ' + df['top11'].astype(str) + ' ' + df['top12'].astype(str)
    return df

In [117]:
perfect_df = correct_customer_id_df.fillna(correct_customer_id_df.mode().iloc[0])


In [119]:
submission_df = join_tops(perfect_df)[["customer_id","prediction"]]

In [120]:
# 終わり！
submission_df

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,372860001 610776002 372860002 568601006 399256...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,372860001 156231001 464297007 610776002 372860...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,372860001 156231001 464297007 610776002 372860...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,720125001 484398001 740519002 730683001 564786...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,372860001 568601006 372860002 610776002 751471...
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,372860001 156231001 464297007 610776002 372860...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,706016001 706016002 759871002 448509014 399223...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,372860001 156231001 464297007 610776002 372860...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,706016001 759871002 706016002 759871001 733749...


In [123]:
# save
submission_df.to_csv("./temp_data/submission.csv", index=False)

# まとめ

## 次やること
- 0417はなぜかcustomer idが足りていないのでsampleと左結語して欠損値を埋める
- 0320フルバージョンで出してこれも結合すること